In [3]:
import datetime as dt  
import pandas as pd
import datetime as dt  	
import numpy as np	  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
import random  		  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 	  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
import pandas as pd  		  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
import util as ut  	
import QLearner as ql 
import indicators as ind
from marketsimcode import compute_portvals

symbol="JPM" 	

hsd=dt.datetime(2003, 1, 1)		  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
hed=dt.datetime(2007, 12, 31)

sd=dt.datetime(2008, 1, 1)		  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
ed=dt.datetime(2009, 12, 31)  

tsd=dt.datetime(2010, 1, 1)		  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
ted=dt.datetime(2011, 12, 31)  
sv=100000 	

In [4]:
syms = [symbol]  	
hdates = pd.date_range(hsd, hed) 
dates = pd.date_range(sd, ed)  	
tdates = pd.date_range(tsd, ted)  	

In [5]:
hprices = ut.get_data(syms, hdates)[syms]
prices= ut.get_data(syms, dates)[syms]
tprices = ut.get_data(syms, tdates)[syms]

hvol = ut.get_data(syms, hdates, colname='Volume')[syms]
vol = ut.get_data(syms, dates, colname='Volume')[syms]
tvol = ut.get_data(syms, tdates, colname='Volume')[syms]

In [6]:
tprices['JPM'].head()

2010-01-04    40.87
2010-01-05    41.67
2010-01-06    41.89
2010-01-07    42.72
2010-01-08    42.62
Name: JPM, dtype: float64

In [44]:
price_change = pd.DataFrame(prices['JPM']- prices['JPM'].shift(1))
price_change = price_change.rename(columns={'JPM': 'pct_chg'})
price_change = price_change.fillna(0)

In [ ]:
tprice_change = pd.DataFrame(tprices['JPM']- tprices['JPM'].shift(1))
tprice_change = tprice_change.rename(columns={'JPM': 'pct_chg'})
tprice_change = tprice_change.fillna(0)

In [7]:
price_range = hprices.describe()
price_range

,JPM
count,1258.000000
mean,33.311335
std,6.925326
min,15.850000
25%,29.022500
50%,31.920000
75%,39.627500
max,47.350000


In [8]:
vol_range = hvol.describe()
vol_range

,JPM
count,1.258000e+03
mean,1.187149e+07
std,6.359128e+06
min,1.347300e+06
25%,8.120175e+06
50%,1.023300e+07
75%,1.340410e+07
max,5.786990e+07


In [9]:
price_thres =  [-np.inf, 
                price_range['JPM']['25%'], 
                price_range['JPM']['75%'],  
                np.inf ]
prices_state = pd.DataFrame(pd.cut(prices['JPM'], bins=price_thres, labels=[1,2,3]))

In [10]:
prices_state = prices_state.rename(columns={'JPM': 'price'})
prices_state.head()

,price
2008-01-02,2
2008-01-03,2
2008-01-04,2
2008-01-07,2
2008-01-08,2


In [11]:
vol_thres =  [-np.inf, 
                vol_range['JPM']['25%'], 
                vol_range['JPM']['75%'],  
                np.inf ]
vol_state = pd.DataFrame(pd.cut(vol['JPM'], bins=vol_thres, labels=[1,2,3]))

In [12]:
vol_state = vol_state.rename(columns={'JPM': 'vol'})
vol_state.head()


,vol
2008-01-02,3
2008-01-03,3
2008-01-04,3
2008-01-07,3
2008-01-08,3


In [51]:
history = dt.timedelta(100)
full_sd = sd - history
df_price_full = ut.get_data([symbol], pd.date_range(full_sd, ed))
df_price_full = df_price_full[syms]
lookback = 60 #3 month trading frequency

bbp = ind.bbp(sd, ed, lookback, df_price_full)

In [52]:
bbp_thres = [-np.inf,0,1,np.inf]
bbp_state = pd.DataFrame(pd.cut(bbp['bbp'], bins=bbp_thres, labels=[1,2,3])) 

In [53]:
bbp_state.head()

,bbp
2008-01-02,2
2008-01-03,2
2008-01-04,2
2008-01-07,2
2008-01-08,1


In [54]:
observations = pd.concat([prices_state, vol_state, bbp_state], axis=1)

In [55]:
observations.head()

,price,vol,bbp
2008-01-02,2,3,2
2008-01-03,2,3,2
2008-01-04,2,3,2
2008-01-07,2,3,2
2008-01-08,2,3,1


In [56]:
observations.iloc[0]['bbp']

2

In [57]:
def convert_state(row): 
    return (row['price'] - 1) * 9 + (row['vol'] -1) * 3 + (row['bbp'] - 1)

In [58]:
state = pd.DataFrame(observations.apply(convert_state, axis=1))

In [59]:
state.describe()

,0
count,505.000000
mean,17.057426
std,6.143027
min,4.000000
25%,16.000000
50%,16.000000
75%,25.000000
max,26.000000


In [60]:
state_price= pd.concat([state, price_change], axis=1).rename(columns={0: "state"})

In [78]:
def get_reward(price_change, oldpos, a): 

    """
    price_change: daily price change 
    oldposition can be 
    1000 share long 
    1000 shares short
    0 shares 
    daily return while action is 0,1,2 correspod to 
    a = -1 short 1000 0,
    a = 0 hold cash 1,
    a = 1 long 1000 2
    return new position, and reward 
    """
    target_shares = (a - 1)* 1000 
    reward = (target_shares - oldpos) * price_change
    return target_shares, reward

In [79]:
def train(state_price, epochs, learner, verbose): 

    startpos = 0
    scores = np.zeros((epochs, 1))
    for epoch in range(1, epochs + 1): 
        total_reward = 0 
        data = state_price.copy()
        oldpos = startpos 

        i = 0 
        while i < len(state_price): 
            
            state = int(data.iloc[i]['state'])

            if i == 0: 
                action = learner.querysetstate(state) # action for first trading day 
            else: 
                action = learner.query(state, reward)
            newpos, reward = get_reward(state_price.iloc[i]['pct_chg'], oldpos, action)
            oldpos = newpos

            if verbose: 
                print(i)
                print(f"action = {action}")	
                print(f"state = {state}")	
                print(f"reward = {reward}")
            
            total_reward += reward
            i += 1 

        scores[epoch - 1, 0] = total_reward
    return np.median(scores)

In [ ]:
def test(state_price, learner, verbose): 

    startpos = 0

    total_reward = 0 
    data = state_price.copy()
    oldpos = startpos 

    i = 0 
    while i < len(state_price): 
        
        state = int(data.iloc[i]['state'])
        action = learner.test_action(state)
        newpos, reward = get_reward(state_price.iloc[i]['pct_chg'], oldpos, action)
        oldpos = newpos

        if verbose: 
            print(i)
            print(f"action = {action}")	
            print(f"state = {state}")	
            print(f"reward = {reward}")
        
        total_reward += reward
        i += 1 

    return total_reward

In [80]:
import random as rand 
def test_code(state_price): 

    data = state_price
    rand.seed(903862041)

    learner = ql.QLearner(
        num_states=27, 
        num_actions=3, 
        alpha=0.2, 
        gamma = 0.9, 
        rar = 0.5,
        radr = 0.99, 
        dyna = 200,
        verbose= False
    )

    epochs = 50
    total_reward = test(state_price, epochs, learner, verbose=False)
    print(f"dyna_score : {total_reward}") 

In [81]:
test_code(state_price)

dyna_score : 71680.00000000007


In [1]:
import importlib
importlib.reload(ql)

NameError: name 'ql' is not defined

In [1]:
list = [1,2,3]
plt.plot(list)

AttributeError: 'list' object has no attribute 'plot'